In [0]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
import os

In [0]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/290")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
pd.set_option('display.max_colwidth',1000)
train = pd.read_csv('train.tsv', sep='\t', header = None)
train.columns = ['ID', 'label', 'statement','subject','speaker','job title','state', 'party','a','b','c','d','e','occasion']
train_new = train.drop(['ID','a','b','c','d','e'],axis = 1)

In [0]:
train_new.head(50)

,label,statement,subject,speaker,job title,state,party,occasion
0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer
1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver
3,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,a news release
4,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN
5,true,The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,a an online opinion-piece
6,barely-true,Jim Dunnam has not lived in the district he represents for years now.,candidates-biography,republican-party-texas,NaN,Texas,republican,a press release.
7,half-true,"I'm the only person on this stage who has worked actively just last year passing, along with Russ Feingold, some of the toughest ethics reform since Watergate.",ethics,barack-obama,President,Illinois,democrat,"a Democratic debate in Philadelphia, Pa."
8,half-true,"However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations Center-Pacific.",jobs,oregon-lottery,NaN,NaN,organization,a website
9,mostly-true,Says GOP primary opponents Glenn Grothman and Joe Leibham cast a compromise vote that cost $788 million in higher electricity costs.,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,an online video


In [0]:
#first split the subject
from keras.preprocessing.sequence import pad_sequences
list_subject = train_new['subject']
list_subject = [str(item).split(',') for item in list_subject]
for i in range(len(list_subject)):
    if len(list_subject[i])<5:
        j = len(list_subject[i])
        while j < 5:
            j+=1
            list_subject[i].append(0)
    else: 
        list_subject[i] = list_subject[i][:5]
list_subject = np.array(list_subject).T

Using TensorFlow backend.


In [0]:
train_new['subject_1'] = list_subject[0]
train_new['subject_2'] = list_subject[1]
train_new['subject_3'] = list_subject[2]
train_new['subject_4'] = list_subject[3]
train_new['subject_5'] = list_subject[4]

In [0]:
train_new.head()

,label,statement,subject,speaker,job title,state,party,occasion,subject_1,subject_2,subject_3,subject_4,subject_5
0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,abortion,0,0,0,0
1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,energy,history,job-accomplishments,0,0
2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver,foreign-policy,0,0,0,0
3,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,a news release,health-care,0,0,0,0
4,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,economy,jobs,0,0,0


In [0]:
#deal with job title
#to word embedding


In [0]:
data_statement=train_new['statement']
data_subject=train_new['subject']
def lemma(msg):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(msg)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output
def clean_text(text):
    text = text.replace("\n"," ")
    text = text.replace("\t"," ")
    text = text.replace("-", ' ')
    text = text.lower()
    return text

In [0]:
train_new['statement_new'] = train_new['statement'].astype(str).str.replace('[^\w\s]','').apply(clean_text)
train_new['statement_new'] = train_new['statement_new'].astype(str).apply(lemma)
# train_new['subject'] = train_new['subject'].apply(lemma)

In [0]:
train_new.head()

,label,statement,subject,speaker,job title,state,party,occasion,subject_1,subject_2,subject_3,subject_4,subject_5,statement_new
0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,abortion,0,0,0,0,say the annies list political group support thirdtrimester abortion on demand
1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,energy,history,job-accomplishments,0,0,when did the decline of coal start it started when natural gas took off that started to begin in president george w bush administration
2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver,foreign-policy,0,0,0,0,hillary clinton agrees with john mccain by voting to give george bush the benefit of the doubt on iran
3,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,a news release,health-care,0,0,0,0,health care reform legislation is likely to mandate free sex change surgery
4,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,economy,jobs,0,0,0,the economic turnaround started at the end of my term


In [0]:
body = train_new.statement_new
bow = {}
for sentence in body:
    words = sentence.split(' ')
    for word in words:
        if not word in bow:
            bow[word] = 1
        else: bow[word] += 1
nbow = {}
i = 0
for item in bow:
    if bow[item] > 5: 
        nbow[item] = i
        i += 1

In [0]:
bow_list = []
for sentence in body:
    temp_bow = [0 for i in range(len(nbow))]
    words = sentence.split(' ')
    for word in words:
        if word in nbow:
            temp_bow[nbow[word]] += 1
    bow_list.append(np.array(temp_bow))


In [0]:
def sentence2bow(sentence, bow):
    sentence_bow = [0 for i in range(len(bow))]
    words = sentence.split(' ')
    for word in words:
        if word in nbow:
            sentence_bow[nbow[word]] += 1
    return sentence_bow

In [0]:
import pickle
output = open('dict.pkl', 'wb')

pickle.dump(nbow, output)

In [0]:
train_new['bow'] = bow_list

In [0]:
train_new.head()

,label,statement,subject,speaker,job title,state,party,occasion,subject_1,subject_2,subject_3,subject_4,subject_5,statement_new,bow
0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,abortion,0,0,0,0,say the annies list political group support thirdtrimester abortion on demand,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,energy,history,job-accomplishments,0,0,when did the decline of coal start it started when natural gas took off that started to begin in president george w bush administration,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver,foreign-policy,0,0,0,0,hillary clinton agrees with john mccain by voting to give george bush the benefit of the doubt on iran,"[0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,a news release,health-care,0,0,0,0,health care reform legislation is likely to mandate free sex change surgery,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
4,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,economy,jobs,0,0,0,the economic turnaround started at the end of my term,"[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [0]:

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def embed_sentences(sentence):
  sentence = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', sentence)
  sentence = tf.convert_to_tensor(sentence)
  sentence = embed(sentence)
  return np.array(sentence[0])


In [0]:

train_new['embed'] = train_new.statement_new.apply(embed_sentences)


In [0]:
train_new.head()


,label,statement,subject,speaker,job title,state,party,occasion,subject_1,subject_2,subject_3,subject_4,subject_5,statement_new,bow,embed
0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,abortion,0,0,0,0,say the annies list political group support thirdtrimester abortion on demand,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0.022711406, -0.070721015, 0.024258291, 0.05122369, 0.050486162, -0.081751965, 0.018096862, -0.05795461, -0.064234026, -0.06661295, 0.044773273, 0.052644804, -0.001384958, -0.07336451, 0.002241063, -0.060802188, -0.06386295, 0.06543958, -0.08240273, 0.057631105, -0.075814955, 0.019270232, -0.03277223, 0.024362752, -0.041681916, 0.063307784, -0.07588432, -0.013455789, -0.050564997, -0.055110287, 0.062241014, 0.001921962, -0.04593654, -0.053481877, 0.06694139, -0.02476327, 0.07366884, 0.051556297, -0.030090515, 0.074420325, 0.08677628, -0.030859442, -0.029963262, -0.038979933, 0.032808416, -0.063631386, 0.055669088, -0.010494819, -0.069971144, -0.02911248, -0.05404047, 0.0820551, -0.030330515, 0.0019633796, -0.021346489, -0.01140548, 0.0026789752, 0.037520874, -0.037687555, -0.02351196, -0.047038525, -0.08552926, -0.03376818, 0.003122482, -0.032020934, 0.050433986, 0.07859019, 0.027825797, -0.024660805, -0.004831191, -0.057517547, 0.06182778, 0.024483318, 0.06803932, -0.03837438, -0..."
1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,energy,history,job-accomplishments,0,0,when did the decline of coal start it started when natural gas took off that started to begin in president george w bush administration,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0.053292155, -0.06711913, 0.05271926, -0.028289724, -0.06787747, 0.05153376, 0.06537125, 0.03165868, 0.055144433, 0.04205743, -0.027324375, -0.010220214, 0.0091147395, 0.026090601, 0.060146343, -0.07189659, 0.011159938, -0.062478412, -0.0057247463, -0.0033264803, -0.06229435, 0.029585432, -0.012299471, 0.057158712, -0.049739935, -0.054678317, 0.023419198, 0.031928204, 0.0051574316, -0.07191321, -0.032635085, -0.03498353, -0.019059287, -0.021823158, 0.012660844, 0.04868699, 0.05213238, -0.060754687, 0.053100977, -0.06868699, 0.07524195, -0.049299978, -0.010711699, -3.2348176e-05, -0.06253115, 0.04112594, 0.012563906, -0.07335271, -0.03869229, -0.06703266, -0.07455943, 0.07578911, -0.015462688, -0.025831543, 0.05485011, 0.04305403, -0.07646268, 0.04816759, 0.018092936, -0.046209257, 0.0749404, -0.07170814, -0.06604494, 0.025213804, -0.039416034, -0.060717624, 0.03821533, 0.06891312, 0.055211663, 0.042363152, -0.026528722, -0.07521935, -0.023826022, -0.015808921, -0.018361045, 0.0344..."
2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver,foreign-policy,0,0,0,0,hillary clinton agrees with john mccain by voting to give george bush the benefit of the doubt on iran,"[0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:
y = train_new.label
x = np.array(train_new.bow)

In [0]:
len(nbow)

2808

In [0]:
#further process on y
d = {'barely-true':1, 'false':0, 'mostly-true':1,'pants-fire':0,'true':1, 'half-true':1}
y_binary = [d[_] for _ in y]

In [0]:
#construct lr model
from sklearn import linear_model
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)
x_train = [_.reshape(len(nbow)) for _ in x_train]
x_test = [_.reshape(len(nbow)) for _ in x_test]

In [0]:
x_train_bin, x_test_bin, y_train_bin, y_test_bin = train_test_split(x, y_binary, test_size=0.2, random_state=100)
x_train_bin = [_.reshape(len(nbow)) for _ in x_train_bin]
x_test_bin = [_.reshape(len(nbow)) for _ in x_test_bin]

In [0]:
lr = linear_model.LogisticRegression(solver = 'newton-cg', multi_class='multinomial')
lr.fit(x_train, y_train)
train_acc=lr.score(x_train,y_train)
print ('Training Accuracy:',train_acc)

test_acc=lr.score(x_test,y_test)
print('Accuracy of the model on unseen test data: ',test_acc)

Training Accuracy: 0.6990966796875
Accuracy of the model on unseen test data:  0.2490234375


In [0]:
lr = linear_model.LogisticRegression(solver = 'newton-cg', multi_class='multinomial')
lr.fit(x_train_bin, y_train_bin)
train_acc=lr.score(x_train_bin,y_train_bin)
print ('Training Accuracy:',train_acc)

test_acc=lr.score(x_test_bin,y_test_bin)
print('Accuracy of the model on unseen test data: ',test_acc)

Training Accuracy: 0.84033203125
Accuracy of the model on unseen test data:  0.65966796875


In [0]:
from sklearn.externals import joblib
joblib.dump(lr,'lr1.joblib')

['lr1.joblib']

In [0]:
from sklearn.metrics import confusion_matrix


In [0]:
y_pred = lr.predict(x_test)
confusion_matrix(y_test_bin, y_pred)


array([[  55,  527],
       [  48, 1418]])

In [0]:
#continue with random forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(bootstrap = True)
rf.fit(x_train, y_train)
train_acc=rf.score(x_train,y_train)
print ('Training Accuracy:',train_acc)

test_acc=rf.score(x_test,y_test)
print('Accuracy of the model on unseen test data: ',test_acc)

KeyboardInterrupt: ignored

In [0]:
rf = RandomForestClassifier(n_estimators = 100, bootstrap = True)
rf.fit(x_train_bin, y_train_bin)
train_acc=rf.score(x_train_bin,y_train_bin)
print ('Training Accuracy:',train_acc)

test_acc=rf.score(x_test_bin,y_test_bin)
print('Accuracy of the model on unseen test data: ',test_acc)

Training Accuracy: 0.9996337890625
Accuracy of the model on unseen test data:  0.7216796875


In [0]:
y_pred = rf.predict(x_test)
confusion_matrix(y_test_bin, y_pred)

array([[  44,  538],
       [  32, 1434]])

In [0]:
joblib.dump(rf,'rf.joblib')

['rf.joblib']

In [0]:

#try to use neural network
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)


2.2.0-rc3


In [0]:

from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop,Adam
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape = (512,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
opt = RMSprop(lr=0.001, decay=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 37,057
Trainable params: 37,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
len(y_train_bin)

8192

In [0]:
his = model.fit(x_train_bin, y_train_bin, batch_size=128, epochs=1, validation_split=0.2)

In [0]:
dat = pd.read_csv('train.csv')
dat.head()

,Unnamed: 0,label,statement,subject,speaker,job title,state,party,occasion,subject_1,subject_2,subject_3,subject_4,subject_5,statement_new,bow,embed
0,0,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,abortion,0,0,0,0,say the annies list political group support thirdtrimester abortion on demand,[1 1 1 ... 0 0 0],[ 0.02271141 -0.07072102 0.02425829 0.05122369 0.05048616 -0.08175196\n 0.01809686 -0.05795461 -0.06423403 -0.06661295 0.04477327 0.0526448\n -0.00138496 -0.07336451 0.00224106 -0.06080219 -0.06386295 0.06543958\n -0.08240273 0.05763111 -0.07581495 0.01927023 -0.03277223 0.02436275\n -0.04168192 0.06330778 -0.07588432 -0.01345579 -0.050565 -0.05511029\n 0.06224101 0.00192196 -0.04593654 -0.05348188 0.06694139 -0.02476327\n 0.07366884 0.0515563 -0.03009051 0.07442033 0.08677628 -0.03085944\n -0.02996326 -0.03897993 0.03280842 -0.06363139 0.05566909 -0.01049482\n -0.06997114 -0.02911248 -0.05404047 0.0820551 -0.03033051 0.00196338\n -0.02134649 -0.01140548 0.00267898 0.03752087 -0.03768755 -0.02351196\n -0.04703853 -0.08552926 -0.03376818 0.00312248 -0.03202093 0.05043399\n 0.07859019 0.0278258 -0.02466081 -0.00483119 -0.05751755 0.06182778\n 0.02448332 0.06803932 -0.03837438 -0.03663548 0.07189956 -0.06146019\n -0.0163452 -0.01946401 0.0221025...
1,1,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,energy,history,job-accomplishments,0,0,when did the decline of coal start it started when natural gas took off that started to begin in president george w bush administration,[0 1 0 ... 0 0 0],[ 5.32921553e-02 -6.71191290e-02 5.27192615e-02 -2.82897241e-02\n -6.78774714e-02 5.15337586e-02 6.53712526e-02 3.16586792e-02\n 5.51444329e-02 4.20574285e-02 -2.73243748e-02 -1.02202138e-02\n 9.11473949e-03 2.60906015e-02 6.01463430e-02 -7.18965903e-02\n 1.11599378e-02 -6.24784119e-02 -5.72474627e-03 -3.32648028e-03\n -6.22943491e-02 2.95854323e-02 -1.22994706e-02 5.71587123e-02\n -4.97399345e-02 -5.46783172e-02 2.34191976e-02 3.19282040e-02\n 5.15743159e-03 -7.19132125e-02 -3.26350853e-02 -3.49835306e-02\n -1.90592874e-02 -2.18231585e-02 1.26608443e-02 4.86869887e-02\n 5.21323793e-02 -6.07546866e-02 5.31009771e-02 -6.86869919e-02\n 7.52419531e-02 -4.92999777e-02 -1.07116988e-02 -3.23481763e-05\n -6.25311509e-02 4.11259383e-02 1.25639057e-02 -7.33527094e-02\n -3.86922918e-02 -6.70326576e-02 -7.45594278e-02 7.57891089e-02\n -1.54626882e-02 -2.58315429e-02 5.48501089e-02 4.30540293e-02\n -7.64626786e-02 4.81675901e-02 1.80929359e-02 -4.62092571e-02\n 7.49...
2,2,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,Denver,foreign-policy,0,0,0,0,hillary clinton agrees with john mccain by voting to give george bush the benefit of the doubt on iran,[0 2 0 ... 0 0 0],[ 0.01505555 -0.06632619 0.02998269 -0.07228719 -0.00261238 0.00891657\n 0.03758741 0.00410642 0.02916694 0.02050534 -0.02433737 -0.00366318\n 0.02903228 0.02482899 0.03865579 -0.07872467 -0.01550823 -0.0364035\n 0.07082216 0.00877049 0.0178363 0.00515133 0.00034676 0.06560361\n -0.01938309 -0.07782158 -0.0430877 -0.0031384 0.02957533 0.01166864\n -0.03401969 -0.00042666 -0.01691376 0.03583942 -0.04990409 0.0730931\n -0.00098597 -0.01269081 -0.05274072 0.07066356 0.07871718 -0.04854183\n -0.06324518 -0.01603976 0.06551801 0.04442103 0.02976227 0.04093299\n 0.05085599 -0.05786348 -0.07871858 -0.02309574 0.03042118 -0.00550548\n 0.01163062 -0.04061506 -0.06642228 0.06103753 -0.03438377 -0.04551324\n 0.07656974 0.03714681 -0.07874832 -0.05028017 0.02990622 0.05905643\n 0.02738581 0.0084508 -0.06530303 -0.0109393 0.01899818 0.03775974\n 0.02870174 0.058

In [0]:
from sklearn.externals import joblib
import pickle
new_sentence = "trump likes china"
inp = open("dict.pkl","rb")
bow = pickle.load(inp)
def sentence2bow(sentence, bow):
    sentence_bow = [0 for i in range(len(bow))]
    words = sentence.split(' ')
    for word in words:
        if word in bow:
            sentence_bow[bow[word]] += 1
    return sentence_bow
logreg = joblib.load("lr1.joblib")

In [0]:
res = sentence2bow(new_sentence,bow)
res

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [0]:
res = logreg.predict(np.array(res).reshape(1,-1))
res

array(['pants-fire'], dtype=object)

In [0]:
np.array(res).reshape(1,-1)

array([[1, 1, 1, ..., 0, 0, 0]])

In [0]:
lr.predict(x_train_bin[0].reshape(1,-1))

array([1])